In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from importlib import reload
import scipy.io as sio
import pkg_resources

%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys

module_path = os.path.abspath(os.path.join('../../'))
if module_path not in sys.path:
    sys.path.append(module_path)

import MultiLearn_GLRM

In [3]:
def load_mat(filename):
    return sio.loadmat(filename)

def label_to_range(label):
    """
    Convert label to range

    Parameters:
    -----------
    label: list of integers
        must be in the form of [1, 1, ..., 1, 2, 2, ..., 2, ..., C, C, ..., C]
        i.e. nondecreasing numbers starting from 1, each element is greater
        than the previous element by at most 1

    Returns:
    --------
    a list of intergers with C + 1 elements, start with 0
    the i-th element is number of elements in label that equals to i
        
    """
    res = [0]
    assert label[0] == 1, 'label must start with 1'
    for i in range(1, len(label)):
        if label[i] == label[i-1]:
            continue
        if label[i] == label[i-1] + 1:
            res.append(i)
        else:
            assert False,\
                ('label[{}] and label[{}] must be equal or two consecutive '
                 'integers, got {} and {}').format(
                     i-1, i, label[i-1], label[i]
                 )
    res.append(len(label))
    return res

def get_block_row(matrix, block_indices, row_range):
    """
    Extract a subset of rows from a matrix

    Parameters:
    -----------
    matrix: 2-d numpy array
        block matrix
    block_indices: integer of list of integers
        indices of extracted blocks, 0-indexed. If indices is a list, return
        the concatenation of all blocks
    row_range: list of intergers
        in the form of [0, c_1, c_1 + c_2, ..., c_1 + c_2 + ... + c_N]
        where c_i is the number of rows in the i-th block

    Returns:
    --------
    a 2-d matrix
    """
    assert matrix.ndim == 2, 'Expect to receive 2-d array input, got shape {}'.format(matrix.shape)
    if isinstance(block_indices, int):
        block_indices = [block_indices]
    # if isinstance(block_indices, (list, np.ndarray, np.generic))
    ids = []
    for i in block_indices:
        ids = ids + list(range(row_range[i], row_range[i+1]))
    return matrix[ids, :].copy()

def get_block_col(matrix, block_indices, col_range):
    """
    Extract a subset of columns from a matrix

    Parameters:
    -----------
    matrix: 2-d numpy array
        block matrix
    block_indices: integer of list of integers
        indices of extracted blocks, 1-indexed. If indices is a list, return
        the concatenation of all blocks
    row_range: list of intergers
        in the form of [0, c_1, c_1 + c_2, ..., c_1 + c_2 + ... + c_N]
        where c_i is the number of columns in the i-th block

    Returns:
    --------
    a 2-d matrix
    """
    assert matrix.ndim == 2, 'Expect to receive 2-d array input, got shape {}'.format(matrix.shape)
    assert matrix.shape[1] == col_range[-1]
    return get_block_row(matrix.T, block_indices, col_range).T

def get_block_col(matrix, block_indices, col_range):
    """
    Extract a subset of columns from a matrix

    Parameters:
    -----------
    matrix: 2-d numpy array
        block matrix
    block_indices: integer of list of integers
        indices of extracted blocks, 1-indexed. If indices is a list, return
        the concatenation of all blocks
    row_range: list of intergers
        in the form of [0, c_1, c_1 + c_2, ..., c_1 + c_2 + ... + c_N]
        where c_i is the number of columns in the i-th block

    Returns:
    --------
    a 2-d matrix
    """
    assert matrix.ndim == 2, 'Expect to receive 2-d array input, got shape {}'.format(matrix.shape)
    assert matrix.shape[1] == col_range[-1]
    return get_block_row(matrix.T, block_indices, col_range).T

def randperm(n):
    """
    get a random permutation of range(n)
    """
    return np.random.permutation(list(range(n)))

def normc(A):
    """
    normalize each column of A to have norm2 = 1
    """
    return A / np.tile(np.sqrt(np.sum(A*A, axis=0)), (A.shape[0], 1))
def vec(A):
    # TODO: rewrite docstrings
    """
    * Syntax: `a = vec(A)`
    * Vectorization of a matrix. This function is a built-in function in some
    recent MATLAB version.
    """
    return A.flatten('C')

In [4]:
# data_fn = pkg_resources.resource_filename('data/myYaleB' + '.mat')
vars_dict = load_mat('./DICTOL_python-master/dictol/data/AR_EigenFace' + '.mat')

Y_train     = normc(vars_dict['tr_dat'])
Y_test      = normc(vars_dict['tt_dat'])
label_train = vec(vars_dict['trls']).astype(int)
label_test  = vec(vars_dict['ttls']).astype(int)

In [5]:
Y_train

array([[-5.03124342e-01, -5.04742099e-01, -5.21088138e-01, ...,
         4.03009662e-01,  5.37699008e-01,  5.19834238e-01],
       [-1.68132971e-02, -6.14705911e-02, -3.86886960e-02, ...,
        -4.50126372e-01,  3.52874799e-01,  4.94850011e-02],
       [-1.40089553e-01, -1.90421827e-01, -1.86757678e-01, ...,
         2.65904848e-01,  1.48112766e-01,  3.16111680e-01],
       ...,
       [-1.51390691e-02,  5.43818716e-03,  2.02274256e-03, ...,
         5.66781084e-03,  3.92088396e-03, -7.44471070e-04],
       [ 7.01069493e-03,  2.04133921e-05, -1.65794632e-03, ...,
        -6.24761669e-03,  2.30501228e-03,  3.25401237e-03],
       [ 2.56713749e-03, -9.75658853e-03, -3.83385540e-03, ...,
        -3.40278595e-03, -9.20898479e-03, -5.85581362e-03]])

In [10]:
len(label_train), len(label_test), Y_train.shape, Y_test.shape

(700, 699, (300, 700), (300, 699))

In [11]:
nb_classes = max(max(label_train), max(label_test))
label_train
one_hot_targets = np.eye(nb_classes)[np.array(label_train)-1]
TRAIN = np.hstack((Y_train.T, one_hot_targets))
np.random.shuffle(TRAIN)

In [14]:
reload(MultiLearn_GLRM)
from MultiLearn_GLRM import Multi_Learn, generate_AXY
from MultiLearn_GLRM.Params.semisupervised_params import Multilearn_GLRM_Semisupervised

embedding_dim = 50
n_class = nb_classes

A_prime, X_prime, Y_prime = generate_AXY.get_semisupervised_glrm_train_form(TRAIN[:,:-nb_classes], TRAIN[:,-nb_classes:], n_class, embedding_dim)
[GLRM_loss_list, X_regulariation_list, Y_regulariation_list, X_grad_restrictions, Y_grad_restrictions] = Multilearn_GLRM_Semisupervised(A_prime, X_prime, Y_prime, n_class)

num_iterations=100000
learning_rate=0.001
result = Multi_Learn.alternating_minimization(A_prime, X_prime, Y_prime, GLRM_loss_list, X_regulariation_list, Y_regulariation_list, X_grad_restrictions, Y_grad_restrictions, num_iterations, learning_rate, n_class)



Metal device set to: Apple M1
epoch: 0
iter: 0 Total loss: tf.Tensor(278050.950695167, shape=(), dtype=float64)


2022-04-01 20:22:15.444931: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-04-01 20:22:15.445100: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


iter: 100 Total loss: tf.Tensor(236135.16184908355, shape=(), dtype=float64)
iter: 200 Total loss: tf.Tensor(196179.28542747494, shape=(), dtype=float64)
iter: 300 Total loss: tf.Tensor(159324.81252822842, shape=(), dtype=float64)
iter: 400 Total loss: tf.Tensor(125400.58995623815, shape=(), dtype=float64)
iter: 500 Total loss: tf.Tensor(94260.25456248817, shape=(), dtype=float64)
iter: 600 Total loss: tf.Tensor(65947.86021424009, shape=(), dtype=float64)
iter: 700 Total loss: tf.Tensor(40668.68296593254, shape=(), dtype=float64)
iter: 800 Total loss: tf.Tensor(19225.766763653493, shape=(), dtype=float64)
iter: 900 Total loss: tf.Tensor(5431.340507303888, shape=(), dtype=float64)
iter: 982 Total loss: tf.Tensor(3724.4327931825064, shape=(), dtype=float64)
iter: 0 Total loss: tf.Tensor(3723.915308248109, shape=(), dtype=float64)
iter: 100 Total loss: tf.Tensor(3340.357063787492, shape=(), dtype=float64)
iter: 200 Total loss: tf.Tensor(3246.425828593472, shape=(), dtype=float64)
iter: 30

In [15]:
train_prob = tf.nn.softmax(np.matmul(result[0], result[1][:,0:n_class])).numpy()
label_train_shuffled =  np.argmax(TRAIN[:,-nb_classes:], axis=1)
accuracy_score(label_train_shuffled, np.argmax(train_prob, axis=1))

0.01

In [17]:
beta = result[1][1:,0:n_class]
Y_final = result[1][1:embedding_dim+1,n_class:]

A = Y_test.T
A_prime =tf.constant(A)


X = np.random.uniform(-1,1, (A.shape[0], embedding_dim))
X_prime = X
X_prime = tf.Variable(X_prime)

Y_prime = Y_final
X_regularization_loss_list, Y_regularization_loss_list = [], []

GLRM_loss_list = [{
    "A_start_row": 0,
    "A_end_row" : A_prime.shape[0],
    "A_start_col" : 0,
    "A_end_col" : A_prime.shape[1],

    "X_start_row": 0,
    "X_end_row" : X_prime.shape[0],
    "X_start_col" : 0,
    "X_end_col" : X_prime.shape[1],

    "Y_start_row": 0,
    "Y_end_row" : Y_prime.shape[0],
    "Y_start_col" : 0,
    "Y_end_col" : Y_prime.shape[1],
    "weight": 1,
    "loss": 'MAE'
}]

X_regularization_loss_list = [
    {
        "X_start_row": 0,
        "X_end_row" : X.shape[0],
        "X_start_col" : 0,
        "X_end_col" : X.shape[1],
        "penalty_type" : 'L2',
        "alpha": 0.001
    },
    {
        "X_start_row": 0,
        "X_end_row" : X.shape[0],
        "X_start_col" : 0,
        "X_end_col" : X.shape[1],
        "penalty_type" : 'L1',
        "alpha": 0.001
    }
]

num_iterations=50000
learning_rate=0.001
result_val = Multi_Learn.predict(A_prime, X_prime, Y_prime, GLRM_loss_list, X_regularization_loss_list, Y_regularization_loss_list, num_iterations, learning_rate)


iter: 0 Total loss: tf.Tensor(263173.32888632466, shape=(), dtype=float64)
iter: 100 Total loss: tf.Tensor(216269.12043286263, shape=(), dtype=float64)
iter: 200 Total loss: tf.Tensor(177081.93543409335, shape=(), dtype=float64)
iter: 300 Total loss: tf.Tensor(143425.2013897097, shape=(), dtype=float64)
iter: 400 Total loss: tf.Tensor(113744.45385915674, shape=(), dtype=float64)
iter: 500 Total loss: tf.Tensor(87545.55420962205, shape=(), dtype=float64)
iter: 600 Total loss: tf.Tensor(64631.32648868706, shape=(), dtype=float64)
iter: 700 Total loss: tf.Tensor(45003.68748837462, shape=(), dtype=float64)
iter: 800 Total loss: tf.Tensor(28876.919175245348, shape=(), dtype=float64)
iter: 900 Total loss: tf.Tensor(16640.25161834191, shape=(), dtype=float64)
iter: 1000 Total loss: tf.Tensor(8878.442087307472, shape=(), dtype=float64)
iter: 1100 Total loss: tf.Tensor(5051.630033402949, shape=(), dtype=float64)
iter: 1200 Total loss: tf.Tensor(3445.713200019267, shape=(), dtype=float64)
iter: 

In [18]:
val_pred = np.hstack((result_val[0], A_prime))
val_pred = np.hstack((val_pred, np.ones(shape=(val_pred.shape[0],1))))
val_pred = np.matmul(val_pred, beta)
val_pred = tf.nn.softmax(val_pred).numpy()
accuracy_score(np.array(label_test)-1, np.argmax(val_pred, axis=1))

# accuracy_score(label_test, val_preds)

# prob_val = np.exp(prob_val)/(1+np.exp(prob_val))
# predictions_val = [1 if x >= 0.5 else 0 for x in prob_val]
# accuracy_score(Y_test, predictions_val)

0.01859799713876967

In [37]:
result[0][:, 1:embedding_dim+1].shape

TensorShape([700, 50])

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(Y_test, predictions_val), confusion_matrix(Y_train, predictions)

In [38]:
from sklearn.linear_model import LogisticRegression

X_train_embedding = result[0][:, 1:embedding_dim+1]
X_train_embedding = X_train_embedding.numpy()

X_test_embedding = result_val[0].numpy()


clf = LogisticRegression().fit(X_train_embedding, label_train)
pred_logistic = clf.predict(X_train_embedding)
pred_logistic_val = clf.predict(X_test_embedding)
accuracy_score(label_train, pred_logistic), accuracy_score(label_test, pred_logistic_val)

(0.18571428571428572, 0.01859799713876967)

In [40]:
X_train_embedding = Y_train.T
X_test_embedding = Y_test.T


clf = LogisticRegression().fit(X_train_embedding, label_train)
pred_logistic = clf.predict(X_train_embedding)
pred_logistic_val = clf.predict(X_test_embedding)
accuracy_score(label_train, pred_logistic), accuracy_score(label_test, pred_logistic_val)

(0.9785714285714285, 0.6738197424892703)

In [ ]:
confusion_matrix(Y_test, pred_logistic_val), confusion_matrix(Y_train, pred_logistic)

In [ ]:
328/(328+236), 328/(328+938)

In [ ]:
1565/(1565+650), 1565/(1565+3805)

In [ ]:
beta[5:]

In [ ]:
A_prime[0]

In [ ]:
x_embeddings

In [ ]:
prob_val[0:10]

In [ ]:
prob[0:10]

In [ ]:
temp = tf.Variable([1,1,1])
tf.where(temp!=1).shape[0]
# tf.where([True, False, False, True]).numpy()

In [ ]:
np.min(val_pred,axis=0)

In [ ]:
np.min(result[0],axis=0)

In [ ]:
beta

In [ ]:
result[0][3:10,:]

In [106]:
loss_info = Y_regularization_loss_list[2]

# A_00, A_10, A_01, A_11 = loss_info['A_start_row'], loss_info['A_end_row'], loss_info['A_start_col'], loss_info['A_end_col']
X_00, X_10, X_01, X_11 = loss_info['X_start_row'], loss_info['X_end_row'], loss_info['X_start_col'], loss_info['X_end_col']
# Y_00, Y_10, Y_01, Y_11 = loss_info['Y_start_row'], loss_info['Y_end_row'], loss_info['Y_start_col'], loss_info['Y_end_col']


# A_prime[A_00:A_10, A_01:A_11]
# X_prime[X_00:X_10, X_01:X_11]
# Y_prime[Y_00:Y_10, Y_01:Y_11]
Y_prime[X_00:X_10, X_01:X_11]


<tf.Tensor: shape=(3,), dtype=float32, numpy=array([0.09003057, 0.24472848, 0.665241  ], dtype=float32)>

In [ ]:
b = 1.
a = 0.
current_min = 1.
current_max = 38.

def scale_class(val):
    return (b-a)*(val-current_min)/(current_max-current_min)

def scale_inv(val):
    return val*(current_max-current_min) / (b-a) +current_min
label_train_arr = np.array(label_train).reshape(-1,1)
vfunc = np.vectorize(scale_class)
label_train_arr_scaled = vfunc(label_train_arr)